In [10]:
import eikon as ek
import pandas as pd
import numpy as np
ek.set_app_key("69de0ce7c5e2468e90a6058ce79f6558ebe3ee30")

In [ ]:
df = ek.get_timeseries(["MSFT.O"],
                       start_date="2016-01-01",
                       end_date="2019-01-10")
df

https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/tutorials#fundamentals-api-rating-transition-matrix
TR.CommonName
TR.TRBCEconomicSector
TR.F.EV enterprise value
EARNINGS - EPS
TR.NumberOfAnalysts
TR.EpsPreSurprisePct(Period=FY1) - EPS predicted surprise
TR.EPSActSurprise(Period=FY0) - EPS actual surprise
----- R.RevenueLIAccuracyScore(Period=FY1) - Revenue SmartEstimate Accuracy score
TR.EPSLIAccuracyScore(Period=FY1) - EPS SmartEstimate Accuracy score


In [ ]:
economic_story = ["TR.F.EV", ""]

In [16]:
ratings_data, err = ek.get_data(instruments=['0#.SPX'],
                                fields=['TR.CommonName',
                                        "TR.TRBCEconomicSector",
                                        'TR.NumberOfAnalysts',
                                        "EARNINGS",
                                        "TR.EPSLIAccuracyScore"])

KeyboardInterrupt: 

In [18]:
ratings_data.head(50)

,Instrument,Company Common Name,TRBC Economic Sector Name,Number of Analysts,EARNINGS,Earnings Per Share - SmartEstimate® Accuracy Score
0,POOL.OQ,Pool Corp,Consumer Cyclicals,11,19.5579,46.0
1,POOL.OQ,,,<NA>,<NA>,34.0
2,POOL.OQ,,,<NA>,<NA>,56.0
3,POOL.OQ,,,<NA>,<NA>,50.0
4,POOL.OQ,,,<NA>,<NA>,51.0
5,POOL.OQ,,,<NA>,<NA>,67.0
6,POOL.OQ,,,<NA>,<NA>,71.0
7,POOL.OQ,,,<NA>,<NA>,55.0
8,CHRW.OQ,CH Robinson Worldwide Inc,Industrials,29,7.2908,63.0
9,CHRW.OQ,,,<NA>,<NA>,35.0


In [12]:
ratings_data.value_counts("TRBC Economic Sector Name")

TRBC Economic Sector Name
Technology                86
Consumer Cyclicals        70
Industrials               70
Healthcare                63
Financials                62
Consumer Non-Cyclicals    39
Real Estate               30
Utilities                 30
Basic Materials           27
Energy                    26
dtype: int64

# Timeseries

In [5]:
Hourly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="hour")
Daily = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="daily")
Weekly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-03-27", interval="weekly")
Monthly = ek.get_timeseries(["MSFT.O"], fields=["Close"], start_date = "2022-03-01", end_date = "2022-05-27", interval="monthly")
Hourly.tail()

MSFT.O,CLOSE
Date,
2022-03-25 20:00:00,303.69
2022-03-25 21:00:00,303.45
2022-03-25 22:00:00,303.48
2022-03-25 23:00:00,303.51
2022-03-26 00:00:00,303.75


In [6]:
#a RIC that you want to get time series
RIC = 'IBM.N'
#start date that you want to get time series; it is in format yyyy-mm-dd
start_date = '2018-01-02'
#get time series till the last market close date using eikon.get_timeseries(..) function
df = ek.get_timeseries(RIC, 'CLOSE', interval='daily', start_date=start_date)
#display time series data frame
df

IBM.N,CLOSE
Date,
2018-01-02,147.334047
2018-01-03,151.383942
2018-01-04,154.450019
2018-01-05,155.204598
2018-01-08,156.140659
...,...
2023-02-06,136.18
2023-02-07,135.84
2023-02-08,135.98


In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from datetime import timedelta, datetime
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller

#function to forecast using ARIMA Model
#input is time series data frame(df) got from eikon.get_timeseries(..)
#and forecast end date which is in format yyyy-mm-dd
def ARIMA_model_forecast(df, forecast_end_date):
    #Set the start and end forecast date time
    #first date is 1 day after time series. The last date is the forecast_end_date parameter
    _e_date = datetime.fromtimestamp(datetime.timestamp(df.index[-1]))
    e_date = _e_date + timedelta(days = 1)
    e_date = e_date.strftime('%Y-%m-%d')

    #find differences of time series which is input of ARIMA model
    df['df_1diff'] = df['CLOSE'].diff()
    df = df[df['df_1diff'].notnull()]
    lag_acf = acf(df['df_1diff'], nlags=50)
    lag_pacf = pacf(df['df_1diff'], nlags=50, method='ols')
    new_spot = df['df_1diff'].resample('D').ffill()
    #call ARIMA model which p=1,d=0 and q=1 with differences of time series
    arma_model = ARIMA(new_spot, order=(1,0,1))
    results = arma_model.fit(disp=0)
    residuals = pd.DataFrame(results.resid)
    #focast time series
    predictions_ARIMA = pd.Series(results.fittedvalues, copy=True)
    predictions_ARIMA_cumsum = predictions_ARIMA.cumsum()
    predictions_ARIMA_final = pd.Series(df['CLOSE'].iloc[0], index=new_spot.index)
    predictions_ARIMA_final = predictions_ARIMA_final.add(predictions_ARIMA_cumsum, fill_value=0)
    new_spot = df.CLOSE.resample('D',label='right').ffill()
    es_model = ExponentialSmoothing(new_spot, trend='add', damped=False, seasonal='mul', seasonal_periods=30)
    es_results = es_model.fit()
    predicted_values = es_model.predict(params=es_results.params, start=e_date, end=forecast_end_date)

    #create data frame from forecast timeseries
    preds = pd.DataFrame(index= pd.date_range(start=e_date, end=forecast_end_date), data=predicted_values, columns=['CLOSE'])

    #Plot graph of past and forecast timeseries
    plt.figure(figsize=(16,7))
    plt.plot(new_spot, label='Actual')
    plt.plot(preds, label='Forecast', color='red')
    plt.legend(loc='best')
    plt.show()
    return preds

#end date that you want to forecast
forecast_end_date = '2020-03-31'
#call the function to forecast time series with ARIMA model
forecast = ARIMA_model_forecast(df,forecast_end_date)
#display forecast time series
forecast

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data). The types seen wereNone and None. The data was
Date
2018-01-03    4.049895
2018-01-04    3.066076
2018-01-05     0.75458
2018-01-06     0.75458
2018-01-07     0.75458
                ...   
2023-02-06       -0.76
2023-02-07       -0.34
2023-02-08        0.14
2023-02-09       -2.23
2023-02-10        1.85
Freq: D, Name: df_1diff, Length: 1865, dtype: Float64
and
 [[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
before. After,
[4.049895360000022 3.0660764399999607 0.7545795600000247 ...
 0.13999999999998636 -2.2299999999999898 1.8499999999999943]
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]].